# Colab setup
## The first step is not neccesary now, as I fixed all the bad errors
1. Install python 3.10. Please reload page (Ctrl + r / F5) after the first cell
2. Then, clone the repo into the colab

In [ ]:
!wget https://github.com/korakot/kora/releases/download/v0.10/py310.sh
!bash ./py310.sh -b -f -p /usr/local
!python -m ipykernel install --name "py310" --user

In [ ]:
!git clone https://github.com/pmozil/discrete_lab_3
%cd discrete_lab_3
!python -m pip install -r requirements.txt

In [ ]:
%cd graphs
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from matplotlib import animation
import queue
import heapq
import time
from tqdm import tqdm

from typing import Tuple

from graph_generation import gnp_random_connected_graph, draw_graph

# Prim's algorithm
Here's a naive implementation using a binary heap. It's logarithmic, at least

In [ ]:
def prim(graph: nx.Graph) -> nx.Graph:
    """
    Create a spanning tree for the graph

    Args:
        graph: nx.Graph - the given graph

    Returns:
        nx.Graph - the spanning tree
    """
    pqueue = []
    nodes = graph.nodes()
    spanning_tree = nx.Graph()
    for edge in graph.edges(data=True):
        heapq.heappush(pqueue, (edge[0], edge[1]))
    visited = [0 for _ in nodes]
    min_edge = heapq.heappop(pqueue)
    spanning_tree.add_edge(*min_edge)
    visited[min_edge[0]] = 1
    visited[min_edge[1]] = 1
    while not pqueue and not all(visited):
        min_edge = heapq.heappop(pqueue)
        if (visited[min_edge[0]]) ^ (visited[min_edge[1]]):
            spanning_tree.add_edge(*min_edge)
            visited[min_edge[0]] = 1
            visited[min_edge[1]] = 1
    return spanning_tree

A bit slow, but it'll do. On to comparing it with the networkx algorithm!

In [ ]:
ITERATIONS = 100
time_taken_networkx = 0
time_taken_native = 0
for i in tqdm(range(ITERATIONS)):
    G = gnp_random_connected_graph(250, 0.2)
    start = time.time()
    nx.minimum_spanning_tree(G, algorithm="prim")
    end = time.time()

    time_taken_networkx += end - start

    start = time.time()
    prim(G)
    end = time.time()

    time_taken_native += end - start

time_taken_native = time_taken_native / ITERATIONS
time_taken_networkx = time_taken_networkx / ITERATIONS

print(f"native: {time_taken_native}s")
print(f"networkx: {time_taken_networkx}s")
print(f"relative: {time_taken_native / time_taken_networkx}")


## TODO: add kruskal

# Floyd's algorithm
On to the Floyd's algorithm now. Here's out code:

In [ ]:
def floyd(graph: nx.Graph) -> Tuple[np.ndarray, np.ndarray]:
    """
    Calculate the distance matrix for a graph

    Args:
        graph: nx.Graph - the directed graph

    Returns:
        tuple[np.ndarray, np.ndarray] - the W and Θ matrixes
    """
    graph = graph.to_directed()
    nodes = np.intp(graph.number_of_nodes())
    distances = nx.to_numpy_array(graph, None, weight="weight", nonedge=np.inf)
    np.fill_diagonal(distances, 0)
    source_nodes = np.zeros_like(distances)

    for k in range(nodes):
        if any(distances[i, i] < 0 for i in range(nodes)):
            print("Negative cycle in the graph!")
            raise ValueError
        dist_fst = distances[k]
        for i in range(nodes):
            dist_snd = distances[i]
            for j in range(nodes):
                new_val = dist_snd[k] + dist_fst[j]
                if new_val < distances[i, j]:
                    distances[i, j] = new_val
                    source_nodes[j, i] = k

    return distances, source_nodes


def floyd_with_numpy(graph: nx.Graph) -> np.ndarray:
    """
    Calculate the distance matrix for a graph, with numpy minimum
        (it should be faster)

    Args:
        graph: nx.Graph - the directed graph

    Returns:
        np.ndarray - the W and Θ matrixes
    """
    graph = graph.to_directed()
    distances = nx.to_numpy_array(graph, None, weight="weight", nonedge=np.inf)
    np.fill_diagonal(distances, 0)

    for i in range(distances.shape[0]):
        distances = np.minimum(
            distances,
            distances[i, :][np.newaxis, :] + distances[:, i][:, np.newaxis],
        )

    return distances

I shamelessly stole the second one from the networkx code. It's ingenious!a
Still like 2x slower than the networkx implementation, even though the code is identicalo
Let's look at the times:

In [ ]:

ITERATIONS = 1000
time_taken_networkx = 0
time_taken_native = 0
for i in tqdm(range(ITERATIONS)):
    G = gnp_random_connected_graph(40, 0.2)
    if nx.negative_edge_cycle(G):
        continue
    start = time.time()
    a = nx.floyd_warshall_numpy(G)
    end = time.time()

    time_taken_networkx += end - start

    start = time.time()
    b = floyd_with_numpy(G)
    end = time.time()

    time_taken_native += end - start

time_taken_native = time_taken_native / ITERATIONS
time_taken_networkx = time_taken_networkx / ITERATIONS

print(f"native: {time_taken_native}s")
print(f"networkx: {time_taken_networkx}s")
print(f"relative: {time_taken_native / time_taken_networkx}")

# Time complexities and plots
Plotting stuff on a cartesian plane is almost like mediation, so let's do that